# RA2CE feature: Losses analysis

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import rasterio

from ra2ce.ra2ce_handler import Ra2ceHandler #import the ra2cehandler to run ra2ce analyses

c:\repos\ra2ce\.env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Single link losses Analysis

In [2]:
root_dir = Path(".\\data\\single_link_losses")

network_ini = root_dir / "network.ini"
analysis_ini = root_dir / "analysis.ini"

handler = Ra2ceHandler(network=network_ini, analysis=analysis_ini)
criticality_analysis_weight = handler.input_config.analysis_config.config_data.indirect[0].weighing.config_value

handler.configure()
handler.run_analysis()

2024-03-29 10:27:17 AM - [ra2ce_logging.py:41] - root - INFO - RA2CE logger initialized.
2024-03-29 10:27:17 AM - [networks.py:186] - root - INFO - Apparently, you already did create a network with ra2ce earlier. Ra2ce will use this: data\single_link_losses\static\output_graph\base_graph.p
2024-03-29 10:27:17 AM - [hazard_overlay.py:108] - root - INFO - Initialized hazard object.
2024-03-29 10:27:17 AM - [hazard_overlay.py:696] - root - INFO - Iterating overlay of GeoPandas Dataframe.
2024-03-29 10:27:17 AM - [hazard_overlay.py:734] - root - INFO - Setting 'base_network_hazard' graph.
2024-03-29 10:27:17 AM - [hazard_intersect_builder_for_tif.py:183] - root - WARNING - Some geometries have NoneType objects (no coordinate information), namely: Empty GeoDataFrame
Columns: [ID, link_type, avgspeed, geometry, node_A, node_B, edge_fid, length]
Index: [].This could be due to segmentation, and might cause an exception in hazard overlay
Network hazard overlay with Inundacion_Fluvial_tr100: 100

[AnalysisResultWrapper(analysis_result=         ID            link_type  \
 0    1357.0           Regional-N   
 1     790.0  Caminos Vecinales-S   
 2    1470.0              Local-S   
 3    1050.0           Regional-S   
 4     934.0           Regional-S   
 ..      ...                  ...   
 726     0.0           Regional-S   
 727     0.0           Regional-S   
 756     0.0              Local-S   
 759     0.0              Local-S   
 812     0.0  Caminos Vecinales-N   
 
                                               geometry  diff_time  detour  \
 0    LINESTRING Z (-69.27096 18.48280 0.00000, -69....       0.19       1   
 1    LINESTRING Z (-71.06260 18.32464 0.00000, -71....        NaN       0   
 2    LINESTRING Z (-71.25666 18.39316 0.00000, -71....       0.11       1   
 3    LINESTRING Z (-71.19405 18.24858 0.00000, -71....       0.32       1   
 4    LINESTRING Z (-71.08824 18.08410 0.00000, -71....       1.54       1   
 ..                                             